In [107]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df1 = pd.read_csv('datasets/players.csv')
df2 = pd.read_csv('datasets/players_teams.csv')
df3 = pd.read_csv('datasets/awards_players.csv')
df4 = pd.read_csv('datasets/teams.csv')
df5 = pd.read_csv('datasets/teams_post.csv')
df6 = pd.read_csv('datasets/coaches.csv')
df7 = pd.read_csv('datasets/series_post.csv')

def corrige_vencedor(teams, series_post):
    # Itera sobre cada rodada ('F', 'CF', 'FR') para ajustar cada fase dos playoffs
    for round_type in ['FR', 'CF', 'F']:
        # Filtra a série específica da rodada
        series_round = series_post[series_post['round'] == round_type]
        
        # Atualiza cada série individualmente
        for _, row in series_round.iterrows():
            year = row['year']
            winner_id = row['tmIDWinner']
            loser_id = row['tmIDLoser']
            
            # Define as colunas que correspondem às rodadas
            if round_type == 'FR':
                round_column = 'firstRound'
            elif round_type == 'CF':
                round_column = 'semis'
            elif round_type == 'F':
                round_column = 'finals'
            
            # Marca o time vencedor como "W" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == winner_id), round_column] = 'W'
            
            # Marca o time perdedor como "L" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == loser_id), round_column] = 'L'
    
    return teams

teams_file = corrige_vencedor(df4, df7)

players_teams_file = df2.drop(columns=['lgID'])
players_file = df1[df1['pos'].notna() & (df1['pos'] != '')]
players_file = players_file.drop(columns=['firstseason', 'lastseason', 'deathDate', 'collegeOther'])
players_file['college'] = players_file['college'].apply(lambda x: 1 if pd.notnull(x) else 0)
merged_df = pd.merge(players_teams_file, players_file, left_on='playerID', right_on='bioID', how='left')
merged_df = merged_df.drop(columns=['bioID'])
awards_players_file = df3.drop(columns=['lgID'])
teams_file = df4.drop(columns=['lgID', 'divID', 'tmORB','tmDRB','tmTRB','opptmORB','opptmDRB','opptmTRB','seeded'])
teams_file['playoff'] = teams_file['playoff'].apply(lambda x: 1 if x=='Y' else 0)

team_post_file = df5.drop(columns=['lgID'])
series_post_file = df7.drop(columns=['lgIDWinner', 'lgIDLoser'])
coaches_file = df6.drop(columns=['lgID'])


awards_grouped = awards_players_file.groupby(['playerID', 'year'])['award'].apply(list).reset_index()
awards_grouped['award'] = awards_grouped['award'].apply(lambda x: x if isinstance(x, list) else [])

merged_df = pd.merge(merged_df, awards_grouped, on=['playerID', 'year'], how='left')
merged_df['award'] = merged_df['award'].apply(lambda x: x if isinstance(x, list) else [])
merged_df = pd.merge(merged_df, teams_file, on=['tmID','year'], how = 'left')

merged_df = merged_df.drop(columns=['franchID', 'name'])

merged_df = pd.merge(merged_df, team_post_file, on=['tmID','year'], how = 'left')

In [108]:
player_count_per_team_year = merged_df.groupby(['year', 'tmID'])['playerID'].nunique().reset_index()
player_count_per_team_year.columns = ['Year', 'Team', 'PlayerCount']

# print(player_count_per_team_year)

# Calculate min, max, and average player count for each year
summary_stats = player_count_per_team_year.groupby('Year')['PlayerCount'].agg(['min', 'max', 'mean']).reset_index()
summary_stats.columns = ['Year', 'MinPlayerCount', 'MaxPlayerCount', 'AvgPlayerCount']

In [109]:
awards_coaches_file = df3.rename(columns={'playerID': 'coachID'})
coach_awards = awards_coaches_file[awards_coaches_file['award'] == 'Coach of the Year']
coach_awards_grouped = coach_awards.groupby(['coachID', 'year'])['award'].apply(list).reset_index()
coaches_file = pd.merge(coaches_file, coach_awards_grouped, on=['coachID', 'year'], how='left')

In [110]:
avg_oRebounds_by_pos = merged_df.groupby('pos')['oRebounds'].mean().reset_index()
avg_dRebounds_by_pos = merged_df.groupby('pos')['dRebounds'].mean().reset_index()

In [111]:
merged_df = merged_df.drop(columns=['rebounds', 'PostRebounds'])
merged_df = merged_df.rename(columns={'GP_x': 'GP_player', 'GP_y': 'GP_team'})

In [112]:
grouped = merged_df.groupby('year').agg({
    'o_pts': 'sum',
    'o_fga': 'sum',
    'o_oreb': 'sum',
    'o_to': 'sum',
    'o_fta': 'sum',
    'o_asts': 'sum',
    'o_fgm' : 'sum',
    'o_ftm': 'sum',
    'o_dreb':'sum',
}).reset_index()

grouped['VOP'] = grouped['o_pts'] / (grouped['o_fga'] - grouped['o_oreb'] + grouped['o_to'] + 0.44 * grouped['o_fta'])
grouped['factor'] = (2 / 3) - (0.5 * (grouped['o_asts'] / grouped['o_fgm'])) / (2 * (grouped['o_fgm'] / grouped['o_ftm']))
grouped['DRB%'] = (grouped['o_dreb'] - grouped['o_oreb']) / grouped['o_dreb']

uPER_df = merged_df.groupby(['playerID', 'year']).agg({
    'minutes': 'sum',     
    'threeMade': 'sum',   
    'assists': 'sum',     
    'fgMade': 'sum',      
    'ftMade': 'sum',      
    'turnovers': 'sum',   
    'fgAttempted': 'sum', 
    'ftAttempted': 'sum', 
    'dRebounds': 'sum',   
    'oRebounds': 'sum',   
    'steals': 'sum',      
    'blocks': 'sum',      
    'PF': 'sum'           
}).reset_index()

uPER_df = uPER_df.merge(grouped[['year', 'VOP', 'factor', 'DRB%']], on='year')

uPER_df['TRB'] = uPER_df['dRebounds'] + uPER_df['oRebounds']

uPER_df['uPER'] = (1 / uPER_df['minutes']) * (
    uPER_df['threeMade'] +
    (2/3) * uPER_df['assists'] +
    (2 - uPER_df['factor'] * (uPER_df['assists'] / uPER_df['fgMade'])) * uPER_df['fgMade'] +
    (uPER_df['ftMade'] * 0.5 * (1 + (1 - (uPER_df['assists'] / uPER_df['fgMade'])) + (2/3) * (uPER_df['assists'] / uPER_df['fgMade']))) -
    uPER_df['VOP'] * uPER_df['turnovers'] -
    uPER_df['VOP'] * uPER_df['DRB%'] * (uPER_df['fgAttempted'] - uPER_df['fgMade']) -
    uPER_df['VOP'] * 0.44 * (0.44 + (0.56 * uPER_df['DRB%'])) * (uPER_df['ftAttempted'] - uPER_df['ftMade']) +
    uPER_df['VOP'] * (1 - uPER_df['DRB%']) * uPER_df['TRB'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['oRebounds'] +
    uPER_df['VOP'] * uPER_df['steals'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['blocks'] -
    uPER_df['PF'] * ((grouped['o_ftm'].mean() / grouped['o_pts'].mean()) - 0.44 * (grouped['o_fta'].mean() / grouped['o_pts'].mean()) * uPER_df['VOP'])
)

lg_uPER = uPER_df.groupby('year')['uPER'].mean().reset_index()
lg_uPER.rename(columns={'uPER': 'lg_uPER'}, inplace=True)

uPER_df = uPER_df.merge(lg_uPER, on='year')

uPER_df['PER'] = uPER_df['uPER'] * (15 / uPER_df['lg_uPER'])


In [113]:
per_to_merge = uPER_df[['playerID', 'year', 'PER']]
merged_df = merged_df.merge(per_to_merge, on=['playerID', 'year'], how='left')

merged_df['TS%'] = (merged_df['points'] / (2 * (merged_df['fgAttempted'] + 0.44 * merged_df['ftAttempted'])))*100
merged_df['eFG%'] = ((merged_df['fgMade'] + 0.5 * merged_df['threeMade']) / merged_df['fgAttempted'])*100
merged_df['stocks'] = (merged_df['steals'] + merged_df['blocks'])
merged_df['dar'] = ((merged_df['steals'] + merged_df['blocks'] + merged_df['oRebounds'] + merged_df['dRebounds'])/merged_df['minutes'])

merged_df['PER'] = merged_df['PER'].fillna(0)
merged_df['TS%'] = merged_df['TS%'].fillna(0)
merged_df['eFG%'] = merged_df['eFG%'].fillna(0)
merged_df['stocks'] = merged_df['stocks'].fillna(0)
merged_df['dar'] = merged_df['dar'].fillna(0)

#Equipas que não foram aos playoffs
merged_df['W'] = merged_df['W'].fillna(0)
merged_df['L'] = merged_df['L'].fillna(0)

In [114]:
def box_plot_for_each_column(dataset):
    numeric_columns = dataset.select_dtypes(include='number')
    if numeric_columns.empty:
        print("No numeric columns found in the dataset.")
    else:
        numeric_columns.boxplot(figsize=(10, 6))
        plt.title("Boxplot for all numeric columns")
        plt.xticks(rotation=45)  # Rotation in x, if necessary
        plt.show()

def pearson_correlation(dataset, size_x, size_y):
    numeric_columns = dataset.select_dtypes(include='number')
    
    if numeric_columns.empty:
        print("Nenhuma coluna numérica encontrada no dataset.")
    else:
        # Correlation matrix
        correlation_matrix = numeric_columns.corr()

        # View
        plt.figure(figsize=(size_x, size_y))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Pearson-correlation')
        plt.show()

def bar_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            value_counts = non_numeric_columns[column].value_counts()
            plt.figure(figsize=(10, 6))
            value_counts.plot(kind='bar')
            plt.title(f"Bar chart for '{column}'")
            plt.xlabel(column)
            plt.ylabel("Count")
            plt.xticks(rotation=45)
            plt.tight_layout()  # Adjust layout to prevent overlap
            plt.show()

# Pie-chart for each column
def pie_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            # Count elements from different categories
            category_counts = dataset[column].value_counts()
            
            # Pie-chart
            plt.figure(figsize=(6, 6))
            category_counts.plot.pie(autopct='%1.1f%%', startangle=140)
            plt.title(f'Distribution of {column}')
            plt.ylabel('')  # Remove o rótulo do eixo Y
            plt.show()


#box_plot_for_each_column(merged_df)
#box_plot_for_each_column(coaches_file)
#box_plot_for_each_column(series_post_file)



In [115]:
#pearson_correlation(merged_df, 100, 80)
#pearson_correlation(coaches_file, 8, 6)
#pearson_correlation(series_post_file, 8, 6)

In [116]:
#bar_chart_for_each_column(merged_df)
#bar_chart_for_each_column(coaches_file)
#bar_chart_for_each_column(series_post_file)

In [117]:
#pie_chart_for_each_column(merged_df)
#pie_chart_for_each_column(coaches_file)
#pie_chart_for_each_column(series_post_file)

In [118]:
def replaceGameResults(column):
    return column.apply(lambda value: '100' if value == 'W' else '010' if value == 'L' else '001')

# Aplicar a função para cada coluna específica
merged_df['firstRound'] = replaceGameResults(merged_df['firstRound'])
merged_df['semis'] = replaceGameResults(merged_df['semis'])
merged_df['finals'] = replaceGameResults(merged_df['finals'])


In [119]:
if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df.to_csv('cleanDatasets/players_and_teams.csv', index=False)
coaches_file.to_csv('cleanDatasets/coaches_and_awards.csv', index=False)
series_post_file.to_csv('cleanDatasets/series_post.csv', index=False)

In [120]:
merged_df2 = merged_df.drop(columns=['minutes','points','threeMade','assists','fgMade','turnovers','fgAttempted','ftAttempted','oRebounds','steals','blocks','PF','o_ftm','o_pts','o_fta','o_pts','o_fga','o_oreb','o_to','o_asts','o_fgm','o_dreb'])
merged_df2 = merged_df2.drop(columns=['GP_player','GS','ftMade','threeAttempted','GP_team'])
merged_df2 = merged_df2.drop(columns=['o_3pm','o_3pa','o_reb','o_pf','o_stl','o_blk','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts'])
merged_df2 = merged_df2.drop(columns=['PostGP','PostGS','PostMinutes','PostPoints','PostoRebounds','PostdRebounds','PostAssists','PostSteals','PostBlocks','PostTurnovers','PostPF','PostfgAttempted','PostfgMade','PostftAttempted','PostftMade','PostthreeAttempted','PostthreeMade','PostDQ'])
merged_df2 = merged_df2.drop(columns=['arena'])

merged_df2['birthDate'] = pd.to_datetime(merged_df['birthDate'], errors='coerce').dt.year
merged_df2 = merged_df2.rename(columns={'birthDate': 'birthYear'})

if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df2.to_csv('cleanDatasets/advancedstatistics.csv', index=False)

In [121]:
#Juntar os treinadores


# Agrupar coaches por year e team, transformando em listas
coaches_grouped = df6.groupby(['year', 'tmID'])['coachID'].apply(list).reset_index()

# Fazer o merge com o ficheiro
merged_df2 = merged_df2.merge(coaches_grouped, on=['year', 'tmID'], how='left')

merged_df2.head(5)

,playerID,year,stint,tmID,dRebounds,dq,pos,height,weight,college,...,min,attend,W,L,PER,TS%,eFG%,stocks,dar,coachID
0,abrossv01w,2,0,MIN,131,2,F,74.0,169,1,...,6475,120607,0.0,0.0,19.293262,48.849265,42.150171,51,0.265957,[aglerbr99w]
1,abrossv01w,3,0,MIN,101,0,F,74.0,169,1,...,6475,139874,0.0,0.0,15.898093,42.774629,40.822785,52,0.245963,"[aglerbr99w, vandehe99w]"
2,abrossv01w,4,0,MIN,97,0,F,74.0,169,1,...,6850,120253,1.0,2.0,18.485960,48.457881,43.684211,55,0.247475,[mcconsu01w]
3,abrossv01w,5,0,MIN,57,0,F,74.0,169,1,...,6850,125097,0.0,2.0,14.545585,45.842753,42.446043,32,0.229437,[mcconsu01w]
4,abrossv01w,6,0,MIN,78,0,F,74.0,169,1,...,6850,113447,0.0,0.0,18.086335,49.331429,45.471014,54,0.207207,[mcconsu01w]


In [122]:
players_stats_prevYear = merged_df2[['playerID','year','PER', 'eFG%', 'TS%','stocks','dRebounds','dar']].drop_duplicates().copy() #TODO Acrescentar aqui mais variaveis

players_stats_prevYear['year'] = players_stats_prevYear['year'] + 1


players_stats_prevYear = players_stats_prevYear.merge(
    merged_df2[['playerID', 'year', 'tmID', 'playoff','coachID']], 
    on=['playerID', 'year'], 
    how='left')

players_stats_prevYear.to_csv('cleanDatasets/players_stats_prevYear.csv', index=False)


In [123]:
#.............................Fazer a media por equipa dos valores mas pegando apenas nos 7 melhores jogadores...............

# Ordenar os jogadores dentro de cada equipe e ano com base no PER (ou outra métrica)
players_stats_prevYear_sorted = players_stats_prevYear.sort_values(by=['tmID', 'year', 'PER'], ascending=[True, True, False])

# Selecionar os 5 melhores jogadores de cada equipe e ano
top_7_players = players_stats_prevYear_sorted.groupby(['tmID', 'year']).head(5)

# Agora, calcular a média das métricas apenas para os melhores jogadores
team_year_stats = top_7_players.groupby(['tmID', 'year', 'playoff']).agg({
    'PER': 'mean',
    'TS%': 'mean',
    'eFG%': 'mean',
    'stocks': 'mean',
    'dRebounds': 'mean',
    'dar': 'mean',
    'coachID': 'first'  # Seleciona o primeiro valor da coluna (presumindo que todos sejam iguais para o grupo)
}).reset_index()

# Salva o novo dataset em um arquivo CSV
team_year_stats.to_csv('cleanDatasets/team_year_stats.csv', index=False)

team_year_stats.head(5)

,tmID,year,playoff,PER,TS%,eFG%,stocks,dRebounds,dar,coachID
0,ATL,9,0.0,19.894077,53.647675,50.207877,26.6,57.2,0.238431,[meadoma99w]
1,ATL,10,1.0,21.751629,55.257063,51.579845,49.6,88.2,0.315709,[meadoma99w]
2,CHA,2,1.0,20.571617,53.362027,47.843485,29.0,52.6,0.217976,[donovan99w]
3,CHA,3,1.0,19.356376,52.945163,47.204217,36.4,73.2,0.236983,[donovan99w]
4,CHA,4,1.0,20.989740,53.544432,46.980211,37.6,75.2,0.213686,[laceytr99w]


In [124]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

#Para depois poder ir buscar as variavies categoricas
team_year_stats_copy = team_year_stats.copy()


# Selecionar as colunas de interesse
features = team_year_stats[['PER', 'TS%', 'eFG%','stocks','dRebounds']] #TODO Acrescentar aqui tambem

# Normalizar os dados usando MinMaxScaler
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Aplicar o PCA
pca = PCA(n_components=3) #TODO mudar aqui o numero de colunas a selecionar
pca.fit(features_scaled)

# Verificar as cargas (coeficientes) dos componentes principais
components = pca.components_

# Baseado nas cargas, você pode decidir as variáveis mais importantes
# Vamos mostrar a importância de cada variável nas componentes principais

# Calcular a soma das cargas absolutas para cada variável
importance = pd.DataFrame(abs(components), columns=['PER', 'TS%', 'eFG%','stocks','dRebounds'], index=['PC1', 'PC2','PC3']) #TODO mudar aqui as variaveis
#TODO meter tantos PC quanto variaves a selecionar
importance_sum = importance.sum(axis=0)

# Selecionar as duas variáveis mais importantes
most_important_features = importance_sum.sort_values(ascending=False).head(3) #TODO mudar aqui o numero de variaveis

# Exibir apenas os nomes das variáveis mais importantes
important_variable_names = most_important_features.index.tolist()

# Inicializar a lista de componentes a serem removidos
components_to_drop = ['PER', 'TS%', 'eFG%','stocks','dRebounds','dar'] #TODO mudar aqui as variavies (TODOS OS ATRIBUTOS AQUI)
#TODO nao esquecer de aqui adicionar as nao numericas, como os coaches

# Remover as variáveis mais importantes da lista de componentes a serem removidos
components_to_drop = [col for col in components_to_drop if col not in important_variable_names]

team_year_stats=team_year_stats.drop(columns=components_to_drop)

#Adicionar as variaveis categoricas, como os coaches, que nao passaram no processo de PCA
team_year_stats['coachID'] = team_year_stats_copy['coachID']


# Exibir o resultado
print(team_year_stats)

    tmID  year  playoff        PER       eFG%  stocks  \
0    ATL     9      0.0  19.894077  50.207877    26.6   
1    ATL    10      1.0  21.751629  51.579845    49.6   
2    CHA     2      1.0  20.571617  47.843485    29.0   
3    CHA     3      1.0  19.356376  47.204217    36.4   
4    CHA     4      1.0  20.989740  46.980211    37.6   
..   ...   ...      ...        ...        ...     ...   
121  WAS     6      0.0  17.387332  46.823234    46.8   
122  WAS     7      1.0  20.574946  51.330317    39.4   
123  WAS     8      0.0  21.686321  55.716693    47.8   
124  WAS     9      0.0  21.568455  45.065012    47.0   
125  WAS    10      1.0  19.989199  51.201160    32.2   

                      coachID  
0                [meadoma99w]  
1                [meadoma99w]  
2                [donovan99w]  
3                [donovan99w]  
4                [laceytr99w]  
..                        ...  
121              [adubari99w]  
122              [adubari99w]  
123  [adubari99w, rollitr01

In [125]:
# Normalizar os dados 

# Selecionar apenas colunas numéricas
# Lista de colunas a normalizar
columns_to_normalize = ['PER', 'eFG%', 'stocks'] #TODO baseado nas colunas selecionadas do dataset de cima

# Aplicar a normalização apenas nas colunas selecionadas
team_year_stats[columns_to_normalize] = team_year_stats[columns_to_normalize].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Verificar o resultado
print(team_year_stats.head())


  tmID  year  playoff       PER      eFG%    stocks       coachID
0  ATL     9      0.0  0.424361  0.642733  0.182965  [meadoma99w]
1  ATL    10      1.0  0.569981  0.695814  0.545741  [meadoma99w]
2  CHA     2      1.0  0.477476  0.551256  0.220820  [donovan99w]
3  CHA     3      1.0  0.382209  0.526523  0.337539  [donovan99w]
4  CHA     4      1.0  0.510254  0.517857  0.356467  [laceytr99w]


In [126]:
#Dividir o dataset para treino, validacao e teste 

dataset_treino = team_year_stats[(team_year_stats['year'] >= 7) & (team_year_stats['year'] <= 9)]
dataset_teste = team_year_stats[team_year_stats['year'] == 10]

In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Features (X) e alvo (y)
X_treino = dataset_treino[important_variable_names] #Antigamente 'PER', 'TS%', 'eFG%'
y_treino = dataset_treino['playoff']

X_teste = dataset_teste[important_variable_names]
y_teste = dataset_teste['playoff']

In [128]:
# Inicializar o modelo de regressão
modelo = RandomForestRegressor(random_state=42)

# Treinar com os dados de treino
modelo.fit(X_treino, y_treino)

y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

print("Mean Squared Error (MSE) na validação:", mse)
print("R² score na validação:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) na validação: 0.17463076923076923
R² score na validação: 0.2621850000000001


/tmp/ipykernel_20671/3804687974.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.85
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.73
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.23
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.71
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.60
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.89
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.77
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.52
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.91
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.31


In [129]:
# Inicializar o modelo de regressão linear
modelo = LinearRegression()

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.2991158585788189
R² score no teste: -0.26376450249550953


/tmp/ipykernel_20671/2857818035.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.724031
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.817805
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.471213
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.446601
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.854950
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],1.348043
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],1.053017
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.490997
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.678520
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.487723


In [130]:
#::::::::::::::::::::Este modelo é uma variante da regressão linear que usa regularização L2 para reduzir overfitting.::::::::::::::::

# Inicializar o modelo de Ridge Regression
modelo = Ridge(alpha=1.0)  # alpha controla o nível de regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.24630559005341002
R² score no teste: -0.04064111797565717


/tmp/ipykernel_20671/11041507.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.684309
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.682951
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.564085
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.531882
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.714708
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.980892
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.823488
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.567930
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.680858
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.554495


In [131]:
# Similar ao Ridge, mas utiliza regularização L1. Tende a eliminar variáveis menos importantes, útil para seleção de features.
# Inicializar o modelo de Lasso Regression
modelo = Lasso(alpha=0.1, random_state=42)  # alpha controla a força da regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.23758751658811145
R² score no teste: -0.00380725758477074


/tmp/ipykernel_20671/613287734.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.585366
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.585366
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.585366
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.585366
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.585366
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.585366
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.585366
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.585366
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.585366
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.585366


In [132]:
# Combina as regularizações L1 (Lasso) e L2 (Ridge).
# Inicializar o modelo de Elastic Net
modelo = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
# `alpha` controla a força total da regularização.
# `l1_ratio` controla a proporção de regularização L1 (Lasso) em relação à L2 (Ridge).

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.23758751658811145
R² score no teste: -0.00380725758477074


/tmp/ipykernel_20671/260154350.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.585366
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.585366
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.585366
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.585366
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.585366
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.585366
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.585366
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.585366
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.585366
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.585366


In [143]:
# Inicializar o modelo de MLP
modelo = MLPRegressor(hidden_layer_sizes=(100, 50),  # Camadas ocultas com 100 e 50 neurônios
                      activation='relu',            # Função de ativação
                      solver='adam',                # Otimizador
                      max_iter=500,                 # Número máximo de iterações
                      random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.2631253217783924
R² score no teste: -0.11170448451370762


/tmp/ipykernel_20671/2522216952.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto,correto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.717456,True
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.764713,False
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.507723,True
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.471178,True
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.800615,True
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],1.088906,True
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.917727,False
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.519572,True
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.675056,True
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.513501,True


In [134]:
import numpy as np

# Inicializar o modelo sequencial
modelo = Sequential()

# Adicionar camadas
modelo.add(Dense(64, activation='relu', input_shape=(X_treino.shape[1],)))  # Primeira camada oculta
modelo.add(Dense(32, activation='relu'))  # Segunda camada oculta
modelo.add(Dense(1))  # Camada de saída (regressão)

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste).flatten()  # Flatten para transformar em 1D

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

/home/emamartins12/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step - loss: 0.4654 - val_loss: 0.5358
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.4332 - val_loss: 0.4977
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.4031 - val_loss: 0.4616
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.3754 - val_loss: 0.4289
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.3505 - val_loss: 0.3986
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.3280 - val_loss: 0.3710
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.3077 - val_loss: 0.3461
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2896 - val_loss: 0.3234
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.2738 - val_loss: 0.3031
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.2601 - val_loss: 0.2855
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.2483 - val_loss: 0.2701
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.2384 - val_loss: 0.2565


/tmp/ipykernel_20671/1782423946.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.661051
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.631602
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.544958
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.522907
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.670135
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.928256
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.774759
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.536967
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.664095
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.522132


In [135]:
# Inicializar o modelo de regressão com Extra Trees
modelo = ExtraTreesRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.22907692307692307
R² score no teste: 0.032150000000000234


/tmp/ipykernel_20671/1732875705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.91
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.69
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.22
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.86
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.45
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.94
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.89
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.77
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.86
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.45


In [136]:
# Inicializar o modelo ElasticNet
modelo = ElasticNet(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
print(dataset_teste.head(15))


Mean Squared Error (MSE) no teste: 0.23758751658811145
R² score no teste: -0.00380725758477074
    tmID  year  playoff       PER      eFG%    stocks  \
1    ATL    10      1.0  0.569981  0.695814  0.545741   
11   CHI    10      0.0  0.434645  0.494162  0.630915   
21   CON    10      0.0  0.631008  0.613080  0.315457   
30   DET    10      1.0  0.667559  0.427084  0.268139   
47   IND    10      1.0  0.478243  0.561043  0.659306   
56   LAS    10      1.0  0.791341  0.557230  1.000000   
67   MIN    10      0.0  0.684573  0.499098  0.772871   
76   NYL    10      0.0  0.516902  0.679304  0.362776   
87   PHO    10      1.0  0.717072  0.688872  0.476341   
98   SAC    10      0.0  0.488918  0.617841  0.359621   
105  SAS    10      1.0  0.381001  0.680233  0.511041   
114  SEA    10      1.0  0.391775  0.553947  0.394322   
125  WAS    10      1.0  0.431818  0.681163  0.271293   

                      coachID  playoff_previsto  
1                [meadoma99w]          0.585366  
11    

/tmp/ipykernel_20671/3962555249.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


In [137]:
# Inicializar o modelo XGBoost Regressor
modelo = xgb.XGBRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.17809230151985936
R² score no teste: 0.2475600260785944


/tmp/ipykernel_20671/1187298798.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],1.003988
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.549502
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.218301
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.832316
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.557558
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.980005
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.793431
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.416923
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.985964
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.277197


In [138]:
# Inicializar o modelo LightGBM Regressor
modelo = lgb.LGBMRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 41, number of used features: 1
[LightGBM] [Info] Start training from score 0.585366
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

/tmp/ipykernel_20671/3421237157.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.649998
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.523811
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.649998
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",0.523811
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.649998
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.649998
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.523811
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.649998
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.649998
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.649998


In [139]:
# Inicializar o modelo de regressão com Gradient Boosting
modelo = GradientBoostingRegressor(random_state=42) #TODO porque foi o melhor modelo até agora

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Critério de correção
# 1. Quando 'playoff' é 1, 'playoff_previsto' deve ser > 0.5
# 2. Quando 'playoff' é 0, 'playoff_previsto' deve ser <= 0.5

# Criar uma coluna indicando se a previsão está correta
dataset_teste['correto'] = ((dataset_teste['playoff'] == 1) & (dataset_teste['playoff_previsto'] > 0.5)) | \
                           ((dataset_teste['playoff'] == 0) & (dataset_teste['playoff_previsto'] <= 0.5))

# Contar o número de previsões corretas
corretos = dataset_teste['correto'].sum()

# Número total de exemplos
total = len(dataset_teste)

# Exibir o resultado
print(f"Previsões corretas: {corretos} de {total} ({(corretos / total) * 100:.2f}%)")


# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.16150240458566234
R² score no teste: 0.3176523406255768
Previsões corretas: 11 de 13 (84.62%)


/tmp/ipykernel_20671/3889390853.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste
/tmp/ipykernel_20671/3889390853.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['correto'] = ((dataset_teste['playoff'] == 1) & (dataset_teste['playoff_previsto'] > 0.5)) | \


,tmID,year,playoff,PER,eFG%,stocks,coachID,playoff_previsto,correto
1,ATL,10,1.0,0.569981,0.695814,0.545741,[meadoma99w],0.968918,True
11,CHI,10,0.0,0.434645,0.494162,0.630915,[keyst99wc],0.823756,False
21,CON,10,0.0,0.631008,0.613080,0.315457,[thibami99w],0.292490,True
30,DET,10,1.0,0.667559,0.427084,0.268139,"[laimbbi01w, mahorri01w]",1.051501,True
47,IND,10,1.0,0.478243,0.561043,0.659306,[dunnli99wc],0.626726,True
56,LAS,10,1.0,0.791341,0.557230,1.000000,[coopemi01w],0.895035,True
67,MIN,10,0.0,0.684573,0.499098,0.772871,[gilloje01w],0.775672,False
76,NYL,10,0.0,0.516902,0.679304,0.362776,"[coylepa99w, donovan99w]",0.457772,True
87,PHO,10,1.0,0.717072,0.688872,0.476341,[gaineco01w],0.949047,True
98,SAC,10,0.0,0.488918,0.617841,0.359621,"[bouceje01w, whisejo99w]",0.424344,True
